profiling does not work with json - skip for now

### TODO
* document cleanup activities
* identify keys / partitions
* identify joins
* write scripts to pull all data to local
* write scripts to pull all data to s3
* determine aws infrastructure
* define etl sequence
* determine etl scripts
* determine airflow pipelines
* identify data quality checks
* define data dictionary

### Done
* add column headers
* for each column
    * determine nulls
    * invalid values
    * text vs numbers
    * missing data
* design data model

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import tqdm

In [2]:
# configurations
github_base_path = '/home/bytze/code/github/udacity-data-engineering-nanodegree/capstone_project'
pd.set_option('display.max_columns', None)

In [3]:
# data imports
github_data_sample = f'{github_base_path}/data/sample'

## Weather Data 

In [4]:
from pandas.io.json import json_normalize
import json

wd = json.load(open(f'{github_data_sample}/weather.json'))
df_weather = json_normalize(wd, 'data').assign(**wd['meta'])

weather_columns={0: 'date', 1: 'max_temp', 2: 'min_temp', 3: 'avg_temp', 
                 4: 'pcpn', 5: 'snow', 6: 'snwd', 'name': 'airport_name',}
df_weather.rename(columns=weather_columns, inplace=True)
weather_profile = df_weather.profile_report(title='Weather_Profile_Report', style={'full_width':True})
weather_profile.to_file(output_file=f'{github_base_path}/data/profiles/weather.html')

In [5]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8036 entries, 0 to 8035
Data columns (total 8 columns):
date            8036 non-null object
max_temp        8036 non-null object
min_temp        8036 non-null object
avg_temp        8036 non-null object
pcpn            8036 non-null object
snow            8036 non-null object
snwd            8036 non-null object
airport_name    8036 non-null object
dtypes: object(8)
memory usage: 502.4+ KB


In [6]:
df_weather.describe(include='all')

,date,max_temp,min_temp,avg_temp,pcpn,snow,snwd,airport_name
count,8036,8036,8036,8036,8036,8036,8036,8036
unique,8036,118,108,216,179,80,24,1
top,1996-09-29,79,62,69.5,0.00,0.0,0,MINNEAPOLIS-ST. PAUL INTERNATIONAL AIRPORT
freq,1,165,168,105,4054,6402,5638,8036


In [7]:
df_weather.head()

,date,max_temp,min_temp,avg_temp,pcpn,snow,snwd,airport_name
0,1987-01-01,26,14,20.0,T,T,T,MINNEAPOLIS-ST. PAUL INTERNATIONAL AIRPORT
1,1987-01-02,30,19,24.5,0.00,0.0,T,MINNEAPOLIS-ST. PAUL INTERNATIONAL AIRPORT
2,1987-01-03,33,18,25.5,0.00,0.0,T,MINNEAPOLIS-ST. PAUL INTERNATIONAL AIRPORT
3,1987-01-04,38,30,34.0,0.00,0.0,T,MINNEAPOLIS-ST. PAUL INTERNATIONAL AIRPORT
4,1987-01-05,35,31,33.0,0.00,0.0,0,MINNEAPOLIS-ST. PAUL INTERNATIONAL AIRPORT


## Weather Data Cleanup

* Remove characters from number fields (pcpn, snow, snwd)
  * M = missing set to 0
  * T = trace amounts set to 0
* change data types of fields

## Airport Data 

In [8]:
df_airport = pd.read_csv(f'{github_data_sample}/airports.csv', header=0, sep=',',
                         quotechar='"')
airport_profile = df_airport.profile_report(title='Airport_Profile_Report', style={'full_width':True})
airport_profile.to_file(output_file=f'{github_base_path}/data/profiles/airport.html')

/home/bytze/code/github/udacity-data-engineering-nanodegree/capstone_project/venv/lib/python3.6/site-packages/pandas_profiling/model/correlations.py:34: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(phi2corr / min((kcorr - 1.0), (rcorr - 1.0)))


In [9]:
df_airport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 7 columns):
iata       3376 non-null object
airport    3376 non-null object
city       3364 non-null object
state      3364 non-null object
country    3376 non-null object
lat        3376 non-null float64
long       3376 non-null float64
dtypes: float64(2), object(5)
memory usage: 184.8+ KB


In [10]:
df_airport.describe(include='all')

,iata,airport,city,state,country,lat,long
count,3376,3376,3364,3364,3376,3376.000000,3376.000000
unique,3376,3245,2675,56,5,NaN,NaN
top,UIZ,Jackson County,Greenville,AK,USA,NaN,NaN
freq,1,5,11,263,3372,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,40.036524,-98.621205
std,NaN,NaN,NaN,NaN,NaN,8.329559,22.869458
min,NaN,NaN,NaN,NaN,NaN,7.367222,-176.646031
25%,NaN,NaN,NaN,NaN,NaN,34.688427,-108.761121
50%,NaN,NaN,NaN,NaN,NaN,39.434449,-93.599425
75%,NaN,NaN,NaN,NaN,NaN,43.372612,-84.137519


In [11]:
df_airport.head()

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944


## Airport Data Cleanup

* remove lines with missing cities - 12
* trim text fields for spaces
* remove records that do not have country = usa
* rename columns: iata to airport code, lat to latitude, long to longitude
* drop any airports we do not have flights for

In [12]:
df_airport[df_airport['city'].isna()]

,iata,airport,city,state,country,lat,long
1136,CLD,MC Clellan-Palomar Airport,NaN,NaN,USA,33.127231,-117.278727
1715,HHH,Hilton Head,NaN,NaN,USA,32.224384,-80.697629
2251,MIB,Minot AFB,NaN,NaN,USA,48.415769,-101.358039
2312,MQT,Marquette County Airport,NaN,NaN,USA,46.353639,-87.395361
2752,RCA,Ellsworth AFB,NaN,NaN,USA,44.145094,-103.103567
2759,RDR,Grand Forks AFB,NaN,NaN,USA,47.961167,-97.401167
2794,ROP,Prachinburi,NaN,NaN,Thailand,14.078333,101.378334
2795,ROR,Babelthoup/Koror,NaN,NaN,Palau,7.367222,134.544167
2900,SCE,University Park,NaN,NaN,USA,40.851206,-77.846302
2964,SKA,Fairchild AFB,NaN,NaN,USA,47.615058,-117.655803


## Flight Data 

In [13]:
df_flight = pd.read_csv(f'{github_data_sample}/1988.csv.bz2', compression='bz2', header=0, sep=',', quotechar='"')
flight_profile = df_flight.profile_report(title='Flight_Profile_Report', style={'full_width':True})
flight_profile.to_file(output_file=f'{github_base_path}/data/profiles/flight.html')

In [14]:
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5202096 entries, 0 to 5202095
Data columns (total 29 columns):
Year                 int64
Month                int64
DayofMonth           int64
DayOfWeek            int64
DepTime              float64
CRSDepTime           int64
ArrTime              float64
CRSArrTime           int64
UniqueCarrier        object
FlightNum            int64
TailNum              float64
ActualElapsedTime    float64
CRSElapsedTime       int64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin               object
Dest                 object
Distance             float64
TaxiIn               float64
TaxiOut              float64
Cancelled            int64
CancellationCode     float64
Diverted             int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtypes: float64(16), int64(10), object(3)
memory usage: 1.1+ GB


In [15]:
df_flight.describe(include='all')

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,5202096.0,5.202096e+06,5.202096e+06,5.202096e+06,5.151933e+06,5.202096e+06,5.137497e+06,5.202096e+06,5202096,5.202096e+06,0.0,5.137497e+06,5.202096e+06,0.0,5.137497e+06,5.151933e+06,5202096,5202096,5.190994e+06,0.0,0.0,5.202096e+06,0.0,5.202096e+06,0.0,0.0,0.0,0.0,0.0
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238,236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ORD,ORD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,753983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271494,274766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1988.0,6.508971e+00,1.575754e+01,3.954361e+00,1.363779e+03,1.357067e+03,1.493592e+03,1.493383e+03,NaN,6.870138e+02,NaN,1.040407e+02,1.039866e+02,NaN,6.547350e+00,6.706768e+00,NaN,NaN,6.015666e+02,NaN,NaN,9.642844e-03,NaN,2.775035e-03,NaN,NaN,NaN,NaN,NaN
std,0.0,3.445201e+00,8.798635e+00,1.987931e+00,4.755135e+02,4.697043e+02,4.937432e+02,4.837890e+02,NaN,5.186402e+02,NaN,6.196058e+01,6.173844e+01,NaN,2.332517e+01,2.177714e+01,NaN,NaN,5.011000e+02,NaN,NaN,9.772339e-02,NaN,5.260547e-02,NaN,NaN,NaN,NaN,NaN
min,1988.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,1.000000e+00,NaN,-5.300000e+02,-5.200000e+01,NaN,-1.185000e+03,-1.000000e+03,NaN,NaN,1.000000e+01,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN
25%,1988.0,4.000000e+00,8.000000e+00,2.000000e+00,9.440000e+02,9.400000e+02,1.118000e+03,1.117000e+03,NaN,2.960000e+02,NaN,6.100000e+01,6.100000e+01,NaN,-5.000000e+00,0.000000e+00,NaN,NaN,2.560000e+02,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN
50%,1988.0,7.000000e+00,1.600000e+01,4.000000e+00,1.340000e+03,1.335000e+03,1.520000e+03,1.519000e+03,NaN,5.780000e+02,NaN,8.500000e+01,8.500000e+01,NaN,2.000000e+00,0.000000e+00,NaN,NaN,4.320000e+02,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN
75%,1988.0,1.000000e+01,2.300000e+01,6.000000e+00,1.748000e+03,1.740000e+03,1.916000e+03,1.911000e+03,NaN,9.380000e+02,NaN,1.320000e+02,1.320000e+02,NaN,1.100000e+01,5.000000e+00,NaN,NaN,8.130000e+02,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN


In [16]:
df_flight.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1988,1,9,6,1348.0,1331,1458.0,1435,PI,942,NaN,70.0,64,NaN,23.0,17.0,SYR,BWI,273.0,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,1988,1,10,7,1334.0,1331,1443.0,1435,PI,942,NaN,69.0,64,NaN,8.0,3.0,SYR,BWI,273.0,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,1988,1,11,1,1446.0,1331,1553.0,1435,PI,942,NaN,67.0,64,NaN,78.0,75.0,SYR,BWI,273.0,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,1988,1,12,2,1334.0,1331,1438.0,1435,PI,942,NaN,64.0,64,NaN,3.0,3.0,SYR,BWI,273.0,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,1988,1,13,3,1341.0,1331,1503.0,1435,PI,942,NaN,82.0,64,NaN,28.0,10.0,SYR,BWI,273.0,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN


## Flight Data Cleanup

* Missing records
  * deptime / arrtime - this is due to a cancellation or the flight was diverted and did not land at its original destination
  
* Keep columns
  * year
  * month
  * dayofmonth
  * dayofweek
  * deptime
  * crsdeptime > schdlddeptime
  * arrtime
  * crsarrtime > schdldarrtime
  * uniquecarrier
  * origin
  * dest
  * distance
  * cancelled
  * diverted

* Keep empty fields, data starts being populated in 2003
  * cancellationcode
  * carrierdelay
  * weatherdelay
  * nasdelay
  * securitydelay
  * lateaircraftdelay

* Drop columns - will calculate our own statistics
  * flightnum
  * tailnum
  * actualelapsedtime
  * crselapsedtime
  * airtime
  * arrdelay
  * depdelay
  * taxiin
  * taxiout
  
* create new columns
  * datetime = month + dayofmonth + timezone
  * depdelay = crsdeptime - deptime
  * arrdelay = crsarrtime - arrtime
  * flighttime = arrtime - deptime
  * schdldflighttime = crsarrtime - crsdeptime
  * 

In [17]:
df_flight[df_flight['ArrTime'].isna()].sample(10)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1215442,1988,3,11,5,1900.0,1900,NaN,2109,AA,635,NaN,NaN,189,NaN,NaN,0.0,MIA,DFW,1121.0,NaN,NaN,0,NaN,1,NaN,NaN,NaN,NaN,NaN
797099,1988,2,16,2,NaN,630,NaN,755,AA,2228,NaN,NaN,85,NaN,NaN,NaN,OAK,LAX,337.0,NaN,NaN,1,NaN,0,NaN,NaN,NaN,NaN,NaN
3388504,1988,8,26,5,1630.0,1442,NaN,1749,AA,482,NaN,NaN,127,NaN,NaN,108.0,DFW,ATL,732.0,NaN,NaN,0,NaN,1,NaN,NaN,NaN,NaN,NaN
1709852,1988,4,18,1,NaN,1735,NaN,1852,US,1919,NaN,NaN,77,NaN,NaN,NaN,LAX,SJC,308.0,NaN,NaN,1,NaN,0,NaN,NaN,NaN,NaN,NaN
1838818,1988,5,16,1,NaN,1930,NaN,2036,EA,373,NaN,NaN,66,NaN,NaN,NaN,GSO,ATL,306.0,NaN,NaN,1,NaN,0,NaN,NaN,NaN,NaN,NaN
459425,1988,2,10,3,NaN,1525,NaN,1914,TW,318,NaN,NaN,169,NaN,NaN,NaN,TUS,STL,1245.0,NaN,NaN,1,NaN,0,NaN,NaN,NaN,NaN,NaN
4294648,1988,10,10,1,NaN,1845,NaN,1920,AS,66,NaN,NaN,35,NaN,NaN,NaN,JNU,SIT,95.0,NaN,NaN,1,NaN,0,NaN,NaN,NaN,NaN,NaN
4182422,1988,10,26,3,NaN,717,NaN,801,EA,85,NaN,NaN,44,NaN,NaN,NaN,PDX,SEA,129.0,NaN,NaN,1,NaN,0,NaN,NaN,NaN,NaN,NaN
5019014,1988,12,10,6,NaN,846,NaN,930,DL,948,NaN,NaN,44,NaN,NaN,NaN,ATL,TYS,152.0,NaN,NaN,1,NaN,0,NaN,NaN,NaN,NaN,NaN
539961,1988,2,3,3,NaN,2040,NaN,2218,EA,627,NaN,NaN,98,NaN,NaN,NaN,ORF,ATL,516.0,NaN,NaN,1,NaN,0,NaN,NaN,NaN,NaN,NaN


In [18]:
df_airport[df_airport['airport'].str.contains("International")].sample(10)

,iata,airport,city,state,country,lat,long
1850,IFP,Laughlin/Bullhead International,Bullhead City,AZ,USA,35.157389,-114.559528
762,ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194
2464,OAK,Metropolitan Oakland International,Oakland,CA,USA,37.721291,-122.220717
2040,LBB,Lubbock International,Lubbock,TX,USA,33.663639,-101.822778
3146,TUL,Tulsa International,Tulsa,OK,USA,36.198372,-95.888242
2768,RIC,Richmond International,Richmond,VA,USA,37.505167,-77.319667
1646,GRR,Kent County International,Grand Rapids,MI,USA,42.880820,-85.522768
931,BDE,Baudette International,Baudette,MN,USA,48.727417,-94.610306
1393,ELP,El Paso International,El Paso,TX,USA,31.806667,-106.377806
1656,GUM,Guam International,Agana,GU,USA,13.483450,-144.795983


## Data Model

### f_flights
* flights_id (integer)
* flight_detail_id (integer)
* airport_depart_id (integer)
* airport_depart_weather_id (integer)
* depart_time_id (integer)
* airport_arrival_id (integer)
* airport_arrival_weather_id (integer)
* arrival_time_id (integer)
* schdld_flight_time_min (integer)
* flight_time_min (integer)
* depart_delay_min (integer)
* arrival_delay_min (integer)

### d_flight_detail
* flight_detail_id (integer)
* carrier (string)
* origin (string)
* dest (string)
* distance (integer)
* schdld_dprt_time (datetime)
* schdld_arrvl_time (datetime)
* cancelled (integer)
* diverted (integer)

### d_time
* time_id (integer)
* date (date)
* datetime (datetime)
* year (integer)
* quarter (integer)
* month (integer)
* day (integer)
* hour (integer)
* minute (integer)

### d_weather
* weather_id (integer)
* date (date)
* max_temp (integer)
* min_temp (integer)
* avg_temp (integer)
* precipitation_in (integer)
* snow_fall_in (integer)
* snow_depth_in (integer)
  
### d_airport
* name (string)
* airport_code (string)
* city (string)
* state (string)
* country (string)
* latitude (float)
* longitude (float)

## Transformations


## SQL Queries